In [1]:
import geopandas as gpd  # For spatial data
import pandas as pd      # For DataFrame and CSV handling
import numpy as np       # For numerical operations
from libpysal import weights  # For spatial weights matrix
from esda import Moran        # For Moran's I spatial autocorrelation test
from spreg import ML_Lag      # For Spatial Lag model (Maximum Likelihood)

# -----------------------------------------------------------
# Load spatial data (GeoPackage) and CSV files
# -----------------------------------------------------------
gdf = gpd.read_file("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\greater_london.gpkg")

tourism_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\londonT&NTlist.csv")
turnover_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\turnover_final_merge.csv")
listings_df = pd.read_csv("E:\\study\\CASAterm1\\CASA0013_FSDS\\group_research\\SAR data\\listings.csv")

# -----------------------------------------------------------
# Rename columns to enable merging by a common key
# -----------------------------------------------------------
gdf = gdf.rename(columns={'geo_code': 'MSOA_CODE'})
turnover_df = turnover_df.rename(columns={'geo_code': 'MSOA_CODE'})

# -----------------------------------------------------------
# Identify tourism-related industry columns
# -----------------------------------------------------------
tourism_categories = ['_retail', '_f&b', '_travel', '_art&ent', '_sp_recrea', '_gambling']
tourism_cols = [col for col in turnover_df.columns if any(cat in col for cat in tourism_categories)]
tourism_cols_numeric = [col for col in tourism_cols if np.issubdtype(turnover_df[col].dtype, np.number)]

# Compute a comprehensive tourism economic indicator by summing over identified columns
turnover_df['tourism_economic_indicator'] = turnover_df[tourism_cols_numeric].sum(axis=1)

# -----------------------------------------------------------
# Convert listings to GeoDataFrame and reproject
# -----------------------------------------------------------
listings_gdf = gpd.GeoDataFrame(
    listings_df,
    geometry=gpd.points_from_xy(listings_df['longitude'], listings_df['latitude']),
    crs="EPSG:4326"
)
listings_gdf = listings_gdf.to_crs(epsg=27700)

# Reproject gdf to match listings_gdf if needed
gdf = gdf.to_crs(epsg=27700)

# Spatial join to assign each Airbnb listing to a MSOA area
listings_with_area = gpd.sjoin(listings_gdf, gdf, how="left", predicate="within")

# Aggregate Airbnb supply by MSOA_CODE
airbnb_supply = listings_with_area.groupby('MSOA_CODE').size().reset_index(name='airbnb_supply')

# -----------------------------------------------------------
# Merge data
# -----------------------------------------------------------
# Select only MSOA_CODE and tourism_economic_indicator from turnover_df
tourism_df_selected = turnover_df[['MSOA_CODE', 'tourism_economic_indicator']]

# Start merging process
merged_df = tourism_df_selected.copy()
merged_df = pd.merge(merged_df, airbnb_supply, on='MSOA_CODE', how='left')
merged_df['airbnb_supply'] = merged_df['airbnb_supply'].fillna(0)

# Merge tourism_df (contains hotspot_binary presumably)
merged_df = pd.merge(merged_df, tourism_df, on='MSOA_CODE', how='left')

# Final merge with spatial data
SARdata = gdf.merge(merged_df, on='MSOA_CODE', how='inner')

# -----------------------------------------------------------
# Create interaction term: airbnb_supply * hotspot_binary
# -----------------------------------------------------------
SARdata['airbnb_interact'] = SARdata['airbnb_supply'] * SARdata['hotspot_binary']

# Compute area in hectares
SARdata['area_m2'] = SARdata.geometry.area
SARdata['area_ha'] = SARdata['area_m2'] / 10000.0

# Compute Airbnb density per hectare
SARdata['airbnb_density_per_ha'] = SARdata['airbnb_supply'] / SARdata['area_ha']

# Prepare variables for the model
X_vars = ['airbnb_density_per_ha', 'airbnb_interact']  # Add control variables if needed
X = SARdata[X_vars].values
y = SARdata['tourism_economic_indicator'].values.reshape(-1, 1)

# -----------------------------------------------------------
# Create and transform spatial weights matrix (Queen contiguity)
# -----------------------------------------------------------
W = weights.contiguity.Queen.from_dataframe(SARdata, use_index=True)
W.transform = 'r'

# Check spatial autocorrelation using Moran's I
mi = Moran(y.flatten(), W)
print("Moran's I:", mi.I)
print("p-value:", mi.p_sim)

# Re-construct W if necessary (if code above changes data)
W = weights.contiguity.Queen.from_dataframe(SARdata, use_index=True)
W.transform = 'r'

# -----------------------------------------------------------
# Run Spatial Lag Model (ML_Lag)
# -----------------------------------------------------------
sar_model = ML_Lag(y, X, w=W, name_y='tourism_economic_indicator',
                   name_x=X_vars, name_w='W', method='full')

print(sar_model.summary)

Moran's I: 0.472779393619594
p-value: 0.001
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :           W
Dependent Variable  :tourism_economic_indicator                Number of Observations:         983
Mean dependent var  :     81.9379                Number of Variables   :           4
S.D. dependent var  :    162.2397                Degrees of Freedom    :         979
Pseudo R-squared    :      0.5771
Spatial Pseudo R-squared:  0.3750
Log likelihood      :  -6022.3516
Sigma-square ML     :  11234.6479                Akaike info criterion :   12052.703
S.E of regression   :    105.9936                Schwarz criterion     :   12072.266

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----